In [1]:
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense
import numpy as np
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [2]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [3]:
# Set features. This will also be used as your x values.
X = df[['koi_period','koi_period_err1','koi_period_err2','koi_time0bk', 'koi_time0bk_err1','koi_time0bk_err2','koi_impact','koi_impact_err1','koi_impact_err2', 'koi_duration', 'koi_duration_err1','koi_duration_err2','koi_depth','koi_depth_err1','koi_depth_err2','koi_prad','koi_prad_err1','koi_prad_err2','koi_teq','koi_insol','koi_insol_err1','koi_insol_err2','koi_model_snr','koi_steff','koi_steff_err1','koi_steff_err2','koi_slogg','koi_slogg_err1','koi_slogg_err2','koi_srad','koi_srad_err1','koi_srad_err2','ra','dec','koi_kepmag']]

# Create a Train Test Split

Use `koi_disposition` for the y values

In [4]:
y = df['koi_disposition']

In [5]:
# Use train_test_split to create training and testing data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [6]:
X_train.head()

,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,koi_impact,koi_impact_err1,koi_impact_err2,koi_duration,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
6122,6.768901,7.380000e-05,-7.380000e-05,133.077240,0.008440,-0.008440,0.150,0.305,-0.150,3.61600,...,-171,4.327,0.153,-0.187,1.125,0.310,-0.207,294.40472,39.351681,14.725
6370,0.733726,6.060000e-06,-6.060000e-06,132.020050,0.007950,-0.007950,0.291,0.193,-0.291,2.30900,...,-175,4.578,0.033,-0.187,0.797,0.211,-0.056,284.50391,42.463860,15.770
2879,7.652707,6.540000e-05,-6.540000e-05,134.460380,0.006190,-0.006190,0.970,0.879,-0.942,79.89690,...,-189,4.481,0.050,-0.200,0.963,0.290,-0.097,295.50211,38.983540,13.099
107,7.953547,1.910000e-05,-1.910000e-05,174.662240,0.001820,-0.001820,0.300,0.145,-0.300,2.63120,...,-85,4.536,0.056,-0.016,0.779,0.023,-0.049,291.15878,40.750271,15.660
29,4.959319,5.150000e-07,-5.150000e-07,172.258529,0.000083,-0.000083,0.831,0.016,-0.004,2.22739,...,-77,4.359,0.110,-0.110,1.082,0.173,-0.130,292.16705,48.727589,15.263


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [7]:
# Scale your data
from sklearn.preprocessing import StandardScaler

# Create a StandardScater model and fit it to the training data
X_scaler = StandardScaler().fit(X_train)
# Transform the training and testing data using the X_scaler

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

C:\Users\Gonzalo\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Gonzalo\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  
C:\Users\Gonzalo\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  if __name__ == '__main__':


In [8]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [9]:
# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

# Train the Model



In [10]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=50, activation='relu', input_dim=35))
model.add(Dense(units=3, activation='softmax'))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [11]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 50)                1800      
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 153       
Total params: 1,953
Trainable params: 1,953
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2
)

Epoch 1/60
5243/5243 - 0s - loss: 0.8649 - acc: 0.6021
Epoch 2/60
5243/5243 - 0s - loss: 0.7200 - acc: 0.6780
Epoch 3/60
5243/5243 - 0s - loss: 0.6676 - acc: 0.7101
Epoch 4/60
5243/5243 - 0s - loss: 0.6375 - acc: 0.7297
Epoch 5/60
5243/5243 - 0s - loss: 0.6169 - acc: 0.7389
Epoch 6/60
5243/5243 - 0s - loss: 0.6011 - acc: 0.7437
Epoch 7/60
5243/5243 - 0s - loss: 0.5883 - acc: 0.7536
Epoch 8/60
5243/5243 - 0s - loss: 0.5789 - acc: 0.7551
Epoch 9/60
5243/5243 - 0s - loss: 0.5691 - acc: 0.7591
Epoch 10/60
5243/5243 - 0s - loss: 0.5614 - acc: 0.7591
Epoch 11/60
5243/5243 - 0s - loss: 0.5560 - acc: 0.7616
Epoch 12/60
5243/5243 - 0s - loss: 0.5494 - acc: 0.7646
Epoch 13/60
5243/5243 - 0s - loss: 0.5466 - acc: 0.7669
Epoch 14/60
5243/5243 - 0s - loss: 0.5401 - acc: 0.7726
Epoch 15/60
5243/5243 - 0s - loss: 0.5358 - acc: 0.7725
Epoch 16/60
5243/5243 - 0s - loss: 0.5330 - acc: 0.7755
Epoch 17/60
5243/5243 - 0s - loss: 0.5300 - acc: 0.7763
Epoch 18/60
5243/5243 - 0s - loss: 0.5268 - acc: 0.7797
E

In [18]:
model_loss, model_accuracy = model.evaluate(
    X_train_scaled, y_train_categorical, verbose=2)
print(
    f"Train Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

5243/5243 - 0s - loss: 0.4597 - acc: 0.8081
Train Neural Network - Loss: 0.45973031389651897, Accuracy: 0.8081251382827759


In [22]:
deep_model = Sequential()
deep_model.add(Dense(units=100, activation='relu', input_dim=35))
deep_model.add(Dense(units=100, activation='relu'))
deep_model.add(Dense(units=3, activation='softmax'))
deep_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 100)               3600      
_________________________________________________________________
dense_6 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_7 (Dense)              (None, 3)                 303       
Total params: 14,003
Trainable params: 14,003
Non-trainable params: 0
_________________________________________________________________


In [23]:
deep_model.compile(optimizer='adam',
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])

deep_model.fit(
    X_test_scaled,
    y_test_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)

Epoch 1/100
1748/1748 - 0s - loss: 0.8780 - acc: 0.6053
Epoch 2/100
1748/1748 - 0s - loss: 0.7344 - acc: 0.6836
Epoch 3/100
1748/1748 - 0s - loss: 0.6736 - acc: 0.7105
Epoch 4/100
1748/1748 - 0s - loss: 0.6392 - acc: 0.7277
Epoch 5/100
1748/1748 - 0s - loss: 0.6053 - acc: 0.7477
Epoch 6/100
1748/1748 - 0s - loss: 0.5816 - acc: 0.7614
Epoch 7/100
1748/1748 - 0s - loss: 0.5648 - acc: 0.7786
Epoch 8/100
1748/1748 - 0s - loss: 0.5469 - acc: 0.7752
Epoch 9/100
1748/1748 - 0s - loss: 0.5324 - acc: 0.7872
Epoch 10/100
1748/1748 - 0s - loss: 0.5286 - acc: 0.7849
Epoch 11/100
1748/1748 - 0s - loss: 0.5104 - acc: 0.7963
Epoch 12/100
1748/1748 - 0s - loss: 0.4958 - acc: 0.8015
Epoch 13/100
1748/1748 - 0s - loss: 0.4929 - acc: 0.7998
Epoch 14/100
1748/1748 - 0s - loss: 0.4853 - acc: 0.8078
Epoch 15/100
1748/1748 - 0s - loss: 0.4752 - acc: 0.8072
Epoch 16/100
1748/1748 - 0s - loss: 0.4626 - acc: 0.8078
Epoch 17/100
1748/1748 - 0s - loss: 0.4553 - acc: 0.8175
Epoch 18/100
1748/1748 - 0s - loss: 0.44

In [24]:
model_loss, model_accuracy = deep_model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(f"Test Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")
#print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
#print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

1748/1748 - 0s - loss: 0.0608 - acc: 0.9949
Test Neural Network - Loss: 0.060757238733277444, Accuracy: 0.9948512315750122


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [26]:
# Create the GridSearch estimator along with a parameter object containing the values to adjust
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
# Function to create model, required for KerasClassifier
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(units=100, activation='relu', input_dim=35))
    model.add(Dense(units=100, activation='relu'))
    model.add(Dense(units=3, activation='softmax'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
# create model
model = KerasClassifier(build_fn=create_model, verbose=2)
# define the grid search parameters
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train_scaled, y_train_categorical)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

C:\Users\Gonzalo\Anaconda3\lib\site-packages\sklearn\externals\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/10
5243/5243 - 2s - loss: 0.4580 - acc: 0.7823
Epoch 2/10
5243/5243 - 1s - loss: 0.3884 - acc: 0.8217
Epoch 3/10
5243/5243 - 1s - loss: 0.3664 - acc: 0.8326
Epoch 4/10
5243/5243 - 1s - loss: 0.3524 - acc: 0.8393
Epoch 5/10
5243/5243 - 1s - loss: 0.3424 - acc: 0.8442
Epoch 6/10
5243/5243 - 1s - loss: 0.3353 - acc: 0.8500
Epoch 7/10
5243/5243 - 1s - loss: 0.3281 - acc: 0.8539
Epoch 8/10
5243/5243 - 2s - loss: 0.3227 - acc: 0.8591
Epoch 9/10
5243/5243 - 1s - loss: 0.3179 - acc: 0.8616
Epoch 10/10
5243/5243 - 1s - loss: 0.3121 - acc: 0.8608
Best: 0.835018 using {'batch_size': 20, 'epochs': 10}
0.831458 (0.002560) with: {'batch_size': 10, 'epochs': 10}
0.818361 (0.001957) with: {'batch_size': 10, 'epochs': 50}
0.810668 (0.002498) with: {'batch_size': 10, 'epochs': 100}
0.835018 (0.007070) with: {'batch_size': 20, 'epochs': 10}
0.826690 (0.001082) with: {'batch_size': 20, 'epochs': 50}
0.809

In [27]:
print(grid.best_params_)
print(grid.best_score_)

{'batch_size': 20, 'epochs': 10}
0.8350181352335057


In [29]:
#Re-run deep_model with hypertuned paramters for neural network model:
deep_model.compile(optimizer='adam',
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])

deep_model.fit(
    X_test_scaled,
    y_test_categorical,
    epochs=10,
    shuffle=True,
    verbose=2
)

Epoch 1/10
1748/1748 - 0s - loss: 0.0856 - acc: 0.9777
Epoch 2/10
1748/1748 - 0s - loss: 0.0761 - acc: 0.9817
Epoch 3/10
1748/1748 - 0s - loss: 0.0729 - acc: 0.9851
Epoch 4/10
1748/1748 - 0s - loss: 0.0647 - acc: 0.9874
Epoch 5/10
1748/1748 - 0s - loss: 0.0666 - acc: 0.9857
Epoch 6/10
1748/1748 - 0s - loss: 0.1189 - acc: 0.9645
Epoch 7/10
1748/1748 - 0s - loss: 0.0638 - acc: 0.9920
Epoch 8/10
1748/1748 - 0s - loss: 0.0609 - acc: 0.9897
Epoch 9/10
1748/1748 - 0s - loss: 0.0597 - acc: 0.9886
Epoch 10/10
1748/1748 - 0s - loss: 0.0598 - acc: 0.9880


In [32]:
print("RESULTS COMPARISSON:")
print("First trained/test deep_model: Accuracy= 0.914  Loss = 0.246")
print("Same deep_model/data with With tuned parameters: Accuracy = 0.988 Loss = 0.0598")

RESULTS COMPARISSON:
First trained/test deep_model: Accuracy= 0.914  Loss = 0.246
Same deep_model/data with With tuned parameters: Accuracy = 0.988 Loss = 0.0598


# Save the Model

In [31]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'GCA_final_with35.sav'
#joblib.dump(model1, filename)